# Anatomical Preprocessing

This notebooks preprocesses anatomical MRI images by executing the following processing steps:

1. Reorient Images to RAS
1. Crop FOV with FSL
1. N4-inhomogenity correction with ANTS
1. GM, WM and CSF Segmentation with SPM
1. Brainmask creation and Brain extraction with FSL
1. Normalization to ICBM template with ANTS

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format. Note that the data should be in a session subfolder:

    dataset
    ├── analysis-anat_specs.json
    └── sub-{sub_id}
        └── ses-{sess_id}
            └── anat
                └── sub-{sub_id}_ses-{sess_id}_{T1_id}.nii.gz

## Execution Specifications

This notebook will extract the relevant processing specifications from the `analysis-anat_specs.json` file in the dataset folder. In the current setup, they are as follows:

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'analysis-anat_specs.json')

with open(spec_file) as f:
    specs = json.load(f)

specs

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject names
subject_list = specs['subject_list']

# List of session names
session_list = specs['session_list']

# Anatomical image identifier
T1_id = specs['T1_id']

# Number of parallel jobs to run
n_proc = specs['n_parallel_jobs']

# Create the Workflow

## Import Modules

In [ ]:
from os.path import join as opj
from nipype import Workflow, Node, Function, IdentityInterface
from nipype.interfaces.image import Reorient
from nipype.interfaces.ants import N4BiasFieldCorrection, Registration
from nipype.interfaces.spm import NewSegment
from nipype.interfaces.fsl import ImageMaths, RobustFOV
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip

# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-dev/spm12_mcr/spm/spm12')

## Relevant Execution Variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/output'

# Location of template brains
template_dir = '/templates/mni_icbm152_nlin_asym_09c/'
brain_template = opj(template_dir, '1.0mm_brain.nii.gz')

## Implement Nodes

In [ ]:
# Reorient anatomical images to RAS
reorient = Node(Reorient(orientation='RAS'), name='reorient')

In [ ]:
# Reduces FOV of images to remove lower head and neck
cropFOV = Node(RobustFOV(output_type='NIFTI_GZ'), name='cropFOV')

In [ ]:
# Corrects bias field
n4 = Node(N4BiasFieldCorrection(dimension=3), name='n4')

In [ ]:
# Gunzips images
gunzip = Node(Gunzip(), name='gunzip')

In [ ]:
# Segments brain into 5 classes (GM, WM, CSF, Skull & Head)
segment = Node(NewSegment(), name='segment')

In [ ]:
# Create brainmask from GM, WM & CSF segmentation
def get_class0(segment_list):
    return segment_list[0][0]

def get_additional_args(segment_list):
    class_1_and_2 = tuple([s[0] for s in segment_list[1:3]])
    return '-add %s -add %s -thr 0.95 -bin' % class_1_and_2

brainmask = Node(ImageMaths(), name='brainmask')

In [ ]:
# Improves brainmask by 1 x erosion, 3 x dilation & filling remaining wholes
def correct_mask(in_file):
    
    import nibabel as nb
    from scipy.ndimage.morphology import (
        binary_fill_holes, binary_dilation, binary_erosion)
    
    img = nb.load(in_file)
    data = img.get_fdata()
    
    data_mask = binary_fill_holes(
        binary_dilation(binary_erosion(
            binary_fill_holes(data), iterations = 1),
                        iterations = 3)).astype('int8')
    new_img = nb.Nifti1Image(data_mask, img.affine)
    out_file = in_file.replace('.nii', '_dil.nii')
    new_img.to_filename(out_file)
    
    return out_file

correct_mask = Node(Function(input_names=['in_file'],
                             output_names=['out_file'],
                             function=correct_mask),
                    name='correct_mask')

In [ ]:
# Apply brainmask to anatomy to extract brain
extract_brain = Node(ImageMaths(op_string='-mul'), name='extract_brain')

In [ ]:
# Normalize anatomy to ICBM template
antsreg = Node(Registration(fixed_image=brain_template,
                            num_threads=n_proc,
                            output_inverse_warped_image=True,
                            output_warped_image=True,

                            collapse_output_transforms=True,
                            dimension=3,
                            float=True,
                            initial_moving_transform_com=True,
                            initialize_transforms_per_stage=False,
                            interpolation='LanczosWindowedSinc',
                            transforms=['Rigid', 'Affine', 'SyN'],
                            transform_parameters=[(0.05,), (0.08,),
                                                  (0.1, 3.0, 0.0)],

                            metric=['Mattes', 'Mattes', 'CC'],
                            metric_weight=[1.0] * 3,
                            radius_or_number_of_bins=[56, 56, 4],
                            sampling_strategy=['Regular', 'Regular', 'None'],
                            sampling_percentage=[0.25, 0.25, 1],
                            number_of_iterations=[[100, 100],
                                                  [100, 100],
                                                  [100, 50, 20, 10]],
                            convergence_threshold=[1e-06] * 3,
                            convergence_window_size=[20, 20, 10],
                            smoothing_sigmas=[[2, 1], [1, 0], [3, 2, 1, 0]],
                            sigma_units=['vox'] * 3,
                            shrink_factors=[[2, 1], [2, 1], [8, 4, 2, 1]],
                            use_estimate_learning_rate_once = [True ,True, True],
                            use_histogram_matching=True,

                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            write_composite_transform=True,
                            terminal_output='file'),
               name='antsreg')

In [ ]:
# Visualize subject to template normalization
def plot_normalization_overlay(in_file, sub, sess, brain_template):

    import nibabel as nb
    from nilearn.plotting import plot_stat_map
    
    # Load GM probability map of TPM.nii
    img = nb.load(brain_template)
    GM_template = nb.Nifti1Image(img.get_fdata(), img.affine, img.header)
    
    title_text = 'sub: %s - sess: %s' % (sub, sess)

    # Plot subject brain on template
    out_file = in_file.replace('.nii.gz', '_overlay.svg')
    plot_stat_map(in_file, title=title_text, colorbar=False, threshold='auto',
                  bg_img=brain_template.replace('brain', 'T1'),
                  display_mode='z', cut_coords=range(-30, 46, 15),
                  output_file=out_file, annotate=False)

    return out_file

vis_norm = Node(Function(input_names=['in_file', 'sub', 'sess', 'brain_template'],
                         output_names=['out_file'],
                         function=plot_normalization_overlay),
                name='vis_norm')
vis_norm.inputs.brain_template = brain_template

In [ ]:
# Visualize brain segmentation
def plot_segmentation(n4, segments, sub, sess):

    import numpy as np
    import nibabel as nb
    from nilearn.plotting import plot_stat_map
    from nilearn.image import coord_transform
    from matplotlib.pyplot import cm
    
    # Plot GM and WM segmentation
    title_text = 'sub: %s - sess: %s' % (sub, sess)
    gm = segments[0][0]
    wm = segments[1][0]

    # Get good cut coordinates
    img_gm = nb.load(gm)
    idx = np.sort(img_gm.get_fdata().nonzero()[-1])
    vox_ids = np.linspace(idx[0], idx[-1], num=10,
                          endpoint=True).astype('int')[2:-2]
    cut_ids = [int(coord_transform(0, 0, r, img_gm.affine)[-1])
               for r in vox_ids]

    # Create segmentation figure
    out_file = n4.replace('.nii.gz', '_overlay.svg')
    display = plot_stat_map(
        gm, cmap=cm.magma, dim=1, colorbar=False, annotate=False, bg_img=n4,
        threshold=0.5, display_mode='z', cut_coords=cut_ids, title=title_text);
    display.add_overlay(wm, threshold=0.5, cmap=cm.hsv)
    display.savefig(out_file)
    display.close()
    
    return out_file

vis_segm = Node(Function(input_names=['n4', 'segments', 'sub', 'sess'],
                         output_names=['out_file'],
                         function=plot_segmentation),
                name='vis_segm')

## Specify Input & Output Stream

In [ ]:
# Iterate over subject and session id
infosource = Node(IdentityInterface(fields=['subject_id', 'session_id']),
                  name='infosource')
infosource.iterables = [('subject_id', subject_list),
                        ('session_id', session_list)]

In [ ]:
# Specify input file location
anat_file = opj('sub-{subject_id}', 'ses-{session_id}', 'anat',
                'sub-{subject_id}_ses-{session_id}_%s.nii.gz' % T1_id)
templates = {'anat': anat_file}

selectfiles = Node(SelectFiles(templates, base_directory='/data'),
                   name='selectfiles')

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [('_session_id_%s_subject_id_%s/' % (sess, sub),
                  'sub-%s/ses-%s_' % (sub, sess))
                 for sess in session_list
                 for sub in subject_list]
substitutions += [('sub-%s_ses-%s' % (sub, sess), '')
                  for sess in session_list
                  for sub in subject_list]
substitutions += [('_%s_ROI_corrected' % T1_id, ''),
                  ('_.nii.gz', '_T1w_corrected.nii.gz'),
                  ('c1', 'seg_gm'),
                  ('c2', 'seg_wm'),
                  ('c3', 'seg_csf'),
                  ('c4', 'seg_skull'),
                  ('c5', 'seg_head'),
                  ('seg_gm_maths_dil', 'brainmask'),
                  ('__maths' , '_brain'),
                  ('__overlay' , '_segmentation')]
datasink.inputs.substitutions = substitutions

## Create Preprocessing Workflow

In [ ]:
# Create anatomical preprocessing workflow
preproc_anat = Workflow(name='preproc_anat')
preproc_anat.base_dir = work_dir

In [ ]:
# Add nodes to workflow and connect them
preproc_anat.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                                 ('session_id', 'session_id')]),

                      # Main part of workflow
                      (selectfiles, reorient, [('anat', 'in_file')]),
                      (reorient, cropFOV, [('out_file', 'in_file')]),
                      (cropFOV, n4, [('out_roi', 'input_image')]),
                      (n4, gunzip, [('output_image', 'in_file')]),
                      (gunzip, segment, [('out_file', 'channel_files')]),
                      (segment, brainmask, [
                          (('native_class_images', get_class0), 'in_file'),
                          (('native_class_images', get_additional_args), 'args')]),
                      (n4, extract_brain, [('output_image', 'in_file')]),
                      (brainmask, correct_mask, [('out_file', 'in_file')]),
                      (correct_mask, extract_brain, [('out_file', 'in_file2')]),
                      (extract_brain, antsreg, [('out_file', 'moving_image')]),

                      # Store main results in datasink
                      (n4, datasink, [('output_image', 'preproc_anat.@n4')]),
                      (segment, datasink, [
                          ('native_class_images', 'preproc_anat.@segment')]),
                      (correct_mask, datasink, [('out_file', 'preproc_anat.@mask')]),
                      (extract_brain, datasink, [('out_file', 'preproc_anat.@brain')]),
                      (antsreg, datasink, [
                          ('warped_image', 'preproc_anat.@warped_image'),
                          ('inverse_warped_image', 'preproc_anat.@inverse_warped_image'),
                          ('composite_transform', 'preproc_anat.@transform'),
                          ('inverse_composite_transform', 'preproc_anat.@inverse_transform')]),

                      # Create and save visual outputs
                      (n4, vis_segm, [('output_image', 'n4')]),
                      (infosource, vis_segm, [('subject_id', 'sub'),
                                              ('session_id', 'sess')]),
                      (segment, vis_segm, [('native_class_images', 'segments')]),

                      (antsreg, vis_norm, [('warped_image', 'in_file')]),
                      (infosource, vis_norm, [('subject_id', 'sub'),
                                              ('session_id', 'sess')]),

                      (vis_norm, datasink, [('out_file', 'viz_anat.@vis_norm')]),
                      (vis_segm, datasink, [('out_file', 'viz_anat.@vis_segm')]),
                      ])

## Visualize Workflow

In [ ]:
# Create preproc_anat output graph
preproc_anat.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph in the notebook
from IPython.display import Image
Image(filename=opj(preproc_anat.base_dir, 'preproc_anat', 'graph.png'))

# Run Workflow

In [ ]:
# Run the workflow in sequential mode
preproc_anat.run('Linear')

In [ ]:
# Save workflow graph visualizations in datasink
preproc_anat.write_graph(graph2use='flat', format='svg', simple_form=True)
preproc_anat.write_graph(graph2use='colored', format='svg', simple_form=True)

from shutil import copyfile
copyfile(opj(preproc_anat.base_dir, 'preproc_anat', 'graph.svg'),
         opj(exp_dir, out_dir, 'preproc_anat', 'graph.svg'))
copyfile(opj(preproc_anat.base_dir, 'preproc_anat', 'graph_detailed.svg'),
         opj(exp_dir, out_dir, 'preproc_anat', 'graph_detailed.svg'));

# Show Created Visualizations in Notebook

In [ ]:
from IPython.display import SVG

In [ ]:
# Visualize the segmentation (segmentation overlayed on T1)
for sub in subject_list:
    for sess in session_list:
        display(SVG(opj(exp_dir, out_dir, 'viz_anat', 'sub-%s' % sub,
                        'ses-%s_segmentation.svg' % sess)))

In [ ]:
# Visualize the normalization (normalized subject overlayed on template)
for sub in subject_list:
    for sess in session_list:
        display(SVG(opj(exp_dir, out_dir, 'viz_anat', 'sub-%s' % sub,
                        'ses-%s_transform_Warped_overlay.svg' % sess)))